In [1]:
from utils.account_status import AccountStatus

account_status = AccountStatus('accounts.csv')

row = account_status.get_account_info_by_plate('Y0V029')

print(row)

plate='Y0V029' ch_code='01RBA2404-4' endo_date='4/3/24' client='ROB AUTO' car_model='2022 MITSUBISHI XPANDER CROSS AT' plate_number_normalized='Y0V029'


In [ ]:
from utils.is_similar_plate import is_similar_plate

plate1 = "ABC1234"
plate2 = "ABC123"

is_similar_plate(plate1, plate2)